# Dynamic drawing from Python to canvas

The [Jupyter Widgets library](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Basics.html) lets us send values from a Python cell to a Javascript cell. We can use these values to create a canvas display that will be dynamically updated whenever we change a `state` object from a Python cell.

Behind the scenes we use `traitlets` which are special data types that can have side effects when their values are modified. Jupyter implements side effects that send the updated values to the browser where they are routed to the appropriate "view", where a view is a JavaScript module and class pairing.

This is our Python side. Here we:

 - Import the widgets library
 - Import our synced data classes (traitlets)
 - Subclass the Jupyter DOMWidget class
 - Define properties on that class which tell the widget library how to find the JavaScript view
 - Define a `state` property which will be synced between Python and JS and can serve as our communication pipe.
     - Note: There's nothing special about the name `state` it could be `potato` or anything you want. The important bit is that it is a traitlet `Dict` with `sync=True`.

In [1]:
import ipywidgets as widgets
from traitlets import Dict, Unicode, validate

class Display(widgets.DOMWidget):
    _view_name = Unicode('Display').tag(sync=True)
    _view_module = Unicode('display').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    state = Dict({"values": []}).tag(sync=True)

This is our JavaScript side. Here we:

 - use `define` from requirejs to create a module.
     - The name of this module must match the `_view_module` in the Python class above.
     - The second argument specifies the list of names of js modules that will be passed to the function defined in the third argument.
 - extend the Jupyter DOMWidgetView to describe what the widget should do. 
     - The name of this class must match the `_view_name` in the Python class above.
 - "export" this new widget by returning it at the end of the function passed to `define`.

In [2]:
%%javascript
// Reset the loader's internal state to forget about the previous definition of the module
require.undef('display');

require.config({
  //Define 3rd party plugins dependencies
  paths: {
    fabric: "https://cdnjs.cloudflare.com/ajax/libs/fabric.js/2.7.0/fabric.min"
  }
});

define('display', ["@jupyter-widgets/base", "fabric"], function(widgets, fabric) {

    var Display = widgets.DOMWidgetView.extend({

        render: function() {
            const canvas = document.createElement('canvas');
            canvas.id = 'canvas';
            canvas.width = 1000;
            canvas.height = 500;
            var ctx = canvas.getContext("2d");
            ctx.fillStyle = "blue";
            ctx.fillRect(0, 0, canvas.width, canvas.height);
            this.el.appendChild(canvas);

            const fabricCanvas = new fabric.Canvas(canvas);

            // Create a starting rect (useful to see something is working)
            const shape = new fabric.Circle({
                top : 100,
                left : 100,
                radius : 20,
                fill : '#5BC8F7'
            });

            fabricCanvas.add(shape);
            
            // Create a list of objects to re-use
            const shapes = [shape];
            
            // Set up our listener
            const onStateChanged = this.handleStateChanged.bind(this, fabricCanvas, shapes);
            this.model.on('change:state', onStateChanged);
        },
        
        handleStateChanged: function(fabricCanvas, shapes) {
            const vals = this.model.get('state').values;
            
            for (let i=0; i < vals.length; i++) {
                let shape;
                // Create a new shape if our list of vals has increased in length
                if (shapes[i] === undefined) {
                    shape = new fabric.Circle({
                        top : 100,
                        left : 100,
                        radius : 20,
                        fill : '#F6C7BE'
                    });
                    fabricCanvas.add(shape);
                    shapes.push(shape);
                } else {
                    shape = shapes[i];
                }
                // Update the shape to the correct location
                const top = vals[i];
                const left = 500 - (i * (shape.radius * 3));
                shape.set({
                    top,
                    left
                });
                
            }
            
            // Render all rects;
            fabricCanvas.renderAll();
        }
    });

    return {
        Display
    };
});

<IPython.core.display.Javascript object>

In [3]:
# Instantiate the Python widget.
display = Display()

In [4]:
# Invoke the __repr__() of the widget which actually causes the JavaScript widget to be drawn.
display

Display(state={'values': []})

In [5]:
# Add some new values to our state and watch our display update live!
import math
from time import sleep
from collections import deque

i = 0
q = deque(maxlen=9)
while True:
    new_state = display.state.copy()
    new_val = (math.sin(i) + 1) * 200
    q.appendleft(new_val)
    new_state['values'] = list(q)
    display.state = new_state
    i += 0.05
    sleep(0.02)

KeyboardInterrupt: 